In [4]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama

In [12]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Write out the following equation using algebraic symbols then solve it."),
    ("system", "Use the format\n\nEQUATION:...\nSOLUTION:...\n\n"),
    ("human", "{equation_statement}"),
])

model = ChatOllama(model="mistral", temperature=0.0)

runnable = {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()

runnable.invoke("x raised to the third plus seven equals 12")

' EQUATION: \\(x^3 + 7 = 12\\)\n\nSOLUTION:\nFirst, subtract 7 from both sides of the equation to isolate the cubed term.\n\n\\(x^3 + 7 = 12 \\)\n\\(x^3 = 5 \\) (subtracting 7 from both sides)\n\nNext, take the cube root of both sides to find the value of x. Since there are three roots for every real number cubed, we need to check the sign of the result to determine which root is valid. In this case, since the left side of the equation is always non-negative (a cube root of a negative number results in a complex number), the solution will be positive.\n\n\\(x^3 = 5 \\)\n\\(x = \\sqrt[3]{5} \\approx 1.612 \\) (taking the real cube root of 5)'

In [14]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop=["SOLUTION"])
    | StrOutputParser()
)

runnable.invoke("x raised to the third plus seven equals 12")

' EQUATION: \\(x^3 + 7 = 12\\)\n\n'

# Attaching `OpenAI functions`

In [15]:
function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}

In [18]:
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Write out the following equation using algebraic symbols then solve it."),
    ("human", "{equation_statement}"),
])

model = ChatOpenAI(model="gpt-4", temperature=0).bind(function_call={"name": "solver"}, functions=[function])
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"equation": "x^3 + 7 = 12",\n"solution": "x = ∛5"\n}', 'name': 'solver'}})

# Attaching `OpenAI tools`

In [20]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [21]:
model = ChatOpenAI(model="gpt-3.5-turbo").bind(tools=tools)
model.invoke("What's the weather in SF, NYC and LA?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_r3AhGTO5CkaEVfVdarxtGWRq', 'function': {'arguments': '{\n  "location": "San Francisco, CA"\n}', 'name': 'get_current_weather'}, 'type': 'function'}]})